In [1]:
# importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Kerala Bus Detalis

In [2]:
# read the csv file
df=pd.read_csv("df_k.csv")
df

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,KSRTC (Kerala) - 52,SWIFT-GARUDA A/C SEATER BUS,12:00,20:20,08h 20m,INR 627,23 Seats available,4.0\n135,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,KSRTC (Kerala) - 1378,AC MULTI AXLE,13:46,21:25,07h 39m,INR 729,25 Seats available,2.7\n36,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,01:00,10h 00m,INR 620,24 Seats available,3.5\n19,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,23:45,08h 20m,INR 729,23 Seats available,3.0\n39,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:00,00:45,08h 45m,INR 567,24 Seats available,3.5\n20,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
270,Madhavi Travels,A/C Sleeper (2+1),18:50,03:00,08h 10m,999,20 Seats available,5.0\n4,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
271,Madhavi Travels,Volvo A/C Semi Sleeper (2+2),23:59,04:30,04h 31m,INR 999,32 Seats available,3.5\n4,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
272,Sonia Travels,A/C Seater/Sleeper (2+1),19:10,02:29,07h 19m,INR 750,35 Seats available,4.1,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode
273,Ubc travels,A/C Seater/Sleeper (2+1),18:00,03:00,09h 00m,INR 900,36 Seats available,3.7,https://www.redbus.in/bus-tickets/kottayam-to-...,Kottayam to Kozhikode


In [3]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [4]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_1 = pd.DataFrame(data)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_1.csv"
df_buses_1.to_csv(path,index=False)

# Andhra Bus Details

In [10]:
# read the csv file
df_1=pd.read_csv("df_A.csv")
df_1

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
4,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
5,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...
6,Tirupati to Bangalore,https://www.redbus.in/bus-tickets/tirupathi-to...
7,Kadapa to Bangalore,https://www.redbus.in/bus-tickets/kadapa-to-ba...
8,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
9,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...


In [11]:
#retrive the bus details
driver_A = webdriver.Chrome()
Bus_names_A = []
Bus_types_A = []
Start_Time_A = []
End_Time_A = []
Ratings_A = []
Total_Duration_A = []
Prices_A = []
Seats_Available_A = []
Route_names = []
Route_links = []

for i,r in df_1.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_A.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_A.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_A.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_A.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_A).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  # Adjust sleep time as needed
        
        new_page_source = driver_A.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_A.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_A.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_A.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_A.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_A.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_A.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_A.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_A.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_A.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_A.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_A.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_A.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_A.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_A.append(ratings.text)
    for price_elem in price:
        Prices_A.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_A.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [12]:
# from list to convert data frame
data_1 = {
    'Bus_name': Bus_names_A,
    'Bus_type': Bus_types_A,
    'Start_time': Start_Time_A,
    'End_time': End_Time_A,
    'Total_duration': Total_Duration_A,
    'Price': Prices_A,
    "Seats_Available":Seats_Available_A,
    "Ratings":Ratings_A,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_2 = pd.DataFrame(data_1)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_2.csv"
df_buses_2.to_csv(path,index=False)

In [13]:
df_buses_2

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,FRESHBUS,Electric A/C Seater (2+2),04:15,11:10,06h 55m,349,27 Seats available,4.7\n463,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
1,FRESHBUS,Electric A/C Seater (2+2),05:40,12:25,06h 45m,349,27 Seats available,4.7\n413,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
2,FRESHBUS,Electric A/C Seater (2+2),11:20,18:50,07h 30m,349,34 Seats available,4.7\n338,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
3,AdYolo Bus,A/C Seater / Sleeper (2+1),22:20,05:29,07h 09m,INR 650,31 Seats available,4.6\n29,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
4,FRESHBUS,Electric A/C Seater (2+2),23:05,06:15,07h 10m,398,32 Seats available,4.6\n528,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
...,...,...,...,...,...,...,...,...,...,...
553,Indu Travels,Non A/C Seater / Sleeper (2+1),22:00,04:30,06h 30m,INR 500,38 Seats available,4.5\n193,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nandyal
554,VEERA BRAHMENDRA TRAVELS,Non A/C Seater / Sleeper (2+1),21:30,04:20,06h 50m,INR 500,26 Seats available,4.2\n299,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nandyal
555,Indu Travels,Non A/C Seater / Sleeper (2+1),21:45,04:20,06h 35m,INR 550,31 Seats available,4.2\n263,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nandyal
556,BRS Travels,Non A/C Seater / Sleeper (2+1),22:30,05:00,06h 30m,INR 550,32 Seats available,4.1\n397,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Nandyal


# Telugana Bus Details

In [9]:
# read the csv file
df_2=pd.read_csv("df_T.csv")
df_2

,Route_name,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
4,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa
5,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
6,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
7,Goa to Belagavi,https://www.redbus.in/bus-tickets/goa-to-belagavi
8,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
9,Solapur to Goa,https://www.redbus.in/bus-tickets/solapur-to-goa


In [10]:
# retrive the bus details
driver_T = webdriver.Chrome()
Bus_names_T = []
Bus_types_T = []
Start_Time_T = []
End_Time_T = []
Ratings_T = []
Total_Duration_T = []
Prices_T = []
Seats_Available_T = []
Route_names = []
Route_links = []

for i,r in df_2.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_T.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_T.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_T.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_T.page_source
        
        ActionChains(driver_T).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_T.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_T.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_T.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_T.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_T.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_T.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_T.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_T.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_T.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_T.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_T.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_T.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_T.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_T.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_T.append(ratings.text)
    for price_elem in price:
        Prices_T.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_T.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [11]:
# from list to convert data frame
data_3 = {
    'Bus_name': Bus_names_T,
    'Bus_type': Bus_types_T,
    'Start_time': Start_Time_T,
    'End_time': End_Time_T,
    'Total_duration': Total_Duration_T,
    'Price': Prices_T,
    "Seats_Available":Seats_Available_T,
    "Ratings":Ratings_T,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_3 = pd.DataFrame(data_3)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_3.csv"
df_buses_3.to_csv(path,index=False)

In [8]:
df_buses_3

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name


# Goa Bus Details

In [12]:
# read the csv file
df_3=pd.read_csv("df_G.csv")
df_3

,Route_name,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
4,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa
5,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
6,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
7,Goa to Belagavi,https://www.redbus.in/bus-tickets/goa-to-belagavi
8,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
9,Solapur to Goa,https://www.redbus.in/bus-tickets/solapur-to-goa


In [19]:
# retrive the bus details
driver_G = webdriver.Chrome()
Bus_names_G = []
Bus_types_G = []
Start_Time_G = []
End_Time_G = []
Ratings_G = []
Total_Duration_G = []
Prices_G = []
Seats_Available_G = []
Route_names = []
Route_links = []

for i,r in df_3.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_G.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_G.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
   
    time.sleep(2)
    scrolling = True
    while scrolling:
        old_page_source = driver_G.page_source
        
        ActionChains(driver_G).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_G.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_G.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_G.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_G.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_G.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_G.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_G.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_G.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_G.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_G.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_G.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_G.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_G.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_G.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_G.append(ratings.text)
    for price_elem in price:
        Prices_G.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_G.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [20]:
# from list to convert data frame
data_4 = {
    'Bus_name': Bus_names_G,
    'Bus_type': Bus_types_G,
    'Start_time': Start_Time_G,
    'End_time': End_Time_G,
    'Total_duration': Total_Duration_G,
    'Price': Prices_G,
    "Seats_Available":Seats_Available_G,
    "Ratings":Ratings_G,
    'Route_link': Route_links,
     'Route_name': Route_names
}

df_buses_4 = pd.DataFrame(data_4)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_4.csv"
df_buses_4.to_csv(path,index=False)

In [21]:
df_buses_4 

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,09:00,10h 30m,INR 900,1 Seat available,4.6\n589,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
1,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:15,07:55,10h 40m,INR 600,32 Seats available,4.5\n588,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
2,Ashray Amrutyog,A/C Sleeper (2+1),22:00,08:00,10h 00m,INR 900,23 Seats available,4.4\n516,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
3,Zingbus Plus,Bharat Benz A/C Sleeper (2+1),23:45,10:55,11h 10m,900,27 Seats available,4.4\n481,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
4,Ashray Travels,Non A/C Seater / Sleeper (2+1),21:20,08:50,11h 30m,INR 500,36 Seats available,4.3\n786,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa
...,...,...,...,...,...,...,...,...,...,...
182,AR Group Vijaylaxmi Travels,VE A/C Sleeper (2+1),19:30,02:45,07h 15m,INR 4999,36 Seats available,4.3,https://www.redbus.in/bus-tickets/goa-to-miraj,Goa to Miraj
183,URL Travels Neeta,A/C Sleeper (2+1),10:30,18:30,08h 00m,INR 3000,23 Seats available,4.1,https://www.redbus.in/bus-tickets/goa-to-miraj,Goa to Miraj
184,Sharma Travels,Bharat Benz A/C Sleeper (2+1),15:00,23:00,08h 00m,INR 800,17 Seats available,3.9,https://www.redbus.in/bus-tickets/goa-to-miraj,Goa to Miraj
185,Venkteshwara Travels,NON A/C Sleeper (2+1),20:30,05:30,09h 00m,INR 499,30 Seats available,3.2,https://www.redbus.in/bus-tickets/goa-to-miraj,Goa to Miraj


# Rajastan Bus Details

In [22]:
# read the csv file
df_4=pd.read_csv("df_R.csv")
df_4

,Route_name,Route_link
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
1,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
2,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
4,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...
7,Kota(Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kota-rajasth...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...


In [23]:
# retrive the bus details
driver_R = webdriver.Chrome()
Bus_names_R = []
Bus_types_R = []
Start_Time_R = []
End_Time_R = []
Ratings_R = []
Total_Duration_R = []
Prices_R = []
Seats_Available_R = []
Route_names_R = []
Route_links_R = []

for i,r in df_4.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_R.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_R.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver_R.page_source
        
        ActionChains(driver_R).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_R.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_R.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_R.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_R.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_R.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_R.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_R.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_R.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_R.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_R.append(bus.text)
        Route_links_R.append(link)
        Route_names_R.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_R.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_R.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_R.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_R.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_R.append(ratings.text)
    for price_elem in price:
        Prices_R.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_R.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [24]:
# from list to convert data frame
data_5 = {
    'Bus_name': Bus_names_R,
    'Bus_type': Bus_types_R,
    'Start_time': Start_Time_R,
    'End_time': End_Time_R,
    'Total_duration': Total_Duration_R,
    'Price': Prices_R,
    "Seats_Available":Seats_Available_R,
    "Ratings":Ratings_R,
    'Route_link': Route_links_R,
    'Route_name': Route_names_R
}

df_buses_5 = pd.DataFrame(data_5)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_5.csv"
df_buses_5.to_csv(path,index=False)

In [25]:
df_buses_5

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Chouhan Tour and Travel,A/C Seater / Sleeper (2+2),01:10,06:00,04h 50m,INR 700,17 Seats available,3.2,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer
1,Shri Ganesh Travels (LR SIYOL),A/C Seater / Sleeper (2+1),01:30,05:00,03h 30m,INR 450,2 Seats available,3.9,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer
2,V k jain travels,NON A/C Seater/ Sleeper (2+1),08:45,13:50,05h 05m,INR 330,36 Seats available,3.0,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer
3,Jain travels regd,NON A/C Sleeper (2+1),11:00,15:30,04h 30m,INR 480,32 Seats available,3.8,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer
4,Jain travels regd,A/C Seater / Sleeper (2+1),14:00,18:00,04h 00m,INR 430,39 Seats available,4.0\n11,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer
...,...,...,...,...,...,...,...,...,...,...
211,Shree Baba Travels,A/C Seater / Sleeper (2+1),02:15,06:00,03h 45m,INR 600,40 Seats available,1.7,https://www.redbus.in/bus-tickets/mathura-to-j...,Mathura to Jaipur (Rajasthan)
212,SHAGUN CARGO & TRAVELS,NON A/C Sleeper (2+1),03:10,07:00,03h 50m,720,31 Seats available,2.7,https://www.redbus.in/bus-tickets/mathura-to-j...,Mathura to Jaipur (Rajasthan)
213,Bhagwati Travel Agency,A/C Seater / Sleeper (2+1),22:40,04:40,06h 00m,449,25 Seats available,2.0\n12,https://www.redbus.in/bus-tickets/mathura-to-j...,Mathura to Jaipur (Rajasthan)
214,GV Tours,A/C Seater / Sleeper (2+1),21:15,04:00,06h 45m,475,40 Seats available,2.3\n20,https://www.redbus.in/bus-tickets/mathura-to-j...,Mathura to Jaipur (Rajasthan)


# South Bengal Bus Details

In [26]:
# read the csv file
df_5=pd.read_csv("df_SB.csv")
df_5

,Route_name,Route_link
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
3,Haldia to Kolkata,https://www.redbus.in/bus-tickets/haldia-to-ko...
4,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
5,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
6,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
7,Kolkata to Midnapore,https://www.redbus.in/bus-tickets/kolkata-to-m...
8,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
9,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...


In [27]:
# retrive the bus details
driver_SB = webdriver.Chrome()
Bus_names_SB = []
Bus_types_SB = []
Start_Time_SB = []
End_Time_SB = []
Ratings_SB = []
Total_Duration_SB = []
Prices_SB = []
Seats_Available_SB = []
Route_names = []
Route_links = []

for i,r in df_5.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_SB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_SB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_SB.page_source
        
        ActionChains(driver_SB).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_SB.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_SB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_SB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_SB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_SB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_SB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_SB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_SB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_SB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_SB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_SB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_SB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_SB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_SB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_SB.append(ratings.text)
    for price_elem in price:
        Prices_SB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_SB.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [28]:
# from list to convert data frame
data_6 = {
    'Bus_name': Bus_names_SB,
    'Bus_type': Bus_types_SB,
    'Start_time': Start_Time_SB,
    'End_time': End_Time_SB,
    'Total_duration': Total_Duration_SB,
    'Price': Prices_SB,
    "Seats_Available":Seats_Available_SB,
    "Ratings":Ratings_SB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_6 = pd.DataFrame(data_6)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_6.csv"
df_buses_6.to_csv(path,index=False)

In [29]:
df_buses_6

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),02:00,06:00,04h 00m,INR 500,8 Seats available,3.6\n85,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
1,Pammi Travels,NON A/C Sleeper (2+1),02:00,06:00,04h 00m,INR 750,22 Seats available,3.6,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
2,Express Line (Karunamoyee),Volvo 9600 Multi Axle Semi-Sleeper (2+2),06:10,09:20,03h 10m,INR 486,33 Seats available,4.6\n273,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
3,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),06:10,09:20,03h 10m,INR 486,26 Seats available,4.0\n7,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
4,Express Line (Karunamoyee),Volvo 9600 Multi Axle Semi-Sleeper (2+2),06:10,09:20,03h 10m,INR 486,33 Seats available,4.5\n11,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
...,...,...,...,...,...,...,...,...,...,...
187,SBSTC-KARUNAMOYEE - JHARGRAM - 08:00 (JHRM DEP...,Non AC Seater (2+3),09:25,00:00,14h 35m,INR 82,51 Seats available,3.9,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Ashari
188,SBSTC-KOLKATA - MIDNAPUR - 09:50 (MID DEPOT) -...,Non AC Seater (2+3),09:50,00:00,14h 10m,INR 82,52 Seats available,3.9,https://www.redbus.in/bus-tickets/kolkata-to-a...,Kolkata to Ashari
189,WBTC (CTC) KOLKATA - ASANSOL - PURULIA (AC VOL...,Volvo AC Seater (2+2),13:45,19:55,06h 10m,INR 595,30 Seats available,3.6\n122,https://www.redbus.in/bus-tickets/kolkata-to-p...,Kolkata to Purulia
190,City Queen Bus Service,Non AC Seater (2+2),21:30,07:30,10h 00m,INR 285,18 Seats available,2.0\n51,https://www.redbus.in/bus-tickets/kolkata-to-p...,Kolkata to Purulia


# Haryana Bus Details

In [30]:
# read the csv file
df_6=pd.read_csv("df_H.csv")
df_6

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
2,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
3,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
4,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...
5,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
6,Chamba (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/chamba-himac...
7,Kangra to Chandigarh,https://www.redbus.in/bus-tickets/kangra-to-ch...
8,Dharamshala (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/dharamshala-...
9,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...


In [31]:
# retrive the bus details
driver_H = webdriver.Chrome()
Bus_names_H = []
Bus_types_H = []
Start_Time_H = []
End_Time_H = []
Ratings_H = []
Total_Duration_H = []
Prices_H = []
Seats_Available_H = []
Route_names_H = []
Route_links_H = []
for i,r in df_6.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_H.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_H.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
         
    scrolling = True
    while scrolling:
        old_page_source = driver_H.page_source
        
        ActionChains(driver_H).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_H.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_H.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_H.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_H.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_H.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_H.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_H.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_H.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_H.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_H.append(bus.text)
        Route_links_H.append(link)
        Route_names_H.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_H.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_H.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_H.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_H.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_H.append(ratings.text)
    for price_elem in price:
        Prices_H.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_H.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [32]:
# from list to convert data frame
data_7 = {
    'Bus_name': Bus_names_H,
    'Bus_type': Bus_types_H,
    'Start_time': Start_Time_H,
    'End_time': End_Time_H,
    'Total_duration': Total_Duration_H,
    'Price': Prices_H,
    "Seats_Available":Seats_Available_H,
    "Ratings":Ratings_H,
    'Route_link': Route_links_H,
    'Route_name': Route_names_H
}

df_buses_7 = pd.DataFrame(data_7)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_7.csv"
df_buses_7.to_csv(path,index=False)

In [33]:
df_buses_7

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Bedi Travels,Volvo A/C Semi Sleeper (2+2),23:00,08:00,09h 00m,495,29 Seats available,4.8\n1030,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
1,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),23:30,08:00,08h 30m,INR 599,32 Seats available,4.6\n797,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
2,Zingbus Plus,Bharat Benz A/C Semi Sleeper (2+2),22:35,08:10,09h 35m,552,31 Seats available,4.6\n509,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
3,AdYolo Bus,Bharat Benz A/C Semi Sleeper (2+2),23:30,09:03,09h 33m,INR 510,33 Seats available,5.0\n5,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
4,Ram Dalal Holidays,Mercedes Multi-Axle Semi Sleeper (2+2),23:00,07:40,08h 40m,570,31 Seats available,4.5\n587,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
...,...,...,...,...,...,...,...,...,...,...
302,HRTC - 700,Himsuta AC Seater Volvo/Scania 2+2,19:00,04:30,09h 30m,INR 1083,30 Seats available,4.6\n21,https://www.redbus.in/bus-tickets/jahoo-to-delhi,Jahu to Delhi
303,HRTC - 749,Ordinary 3+2 Non AC Seater,19:00,05:30,10h 30m,INR 611,37 Seats available,4.3\n8,https://www.redbus.in/bus-tickets/jahoo-to-delhi,Jahu to Delhi
304,HRTC - 750,A/C Executive (2+3),19:30,06:00,10h 30m,INR 696,44 Seats available,4.1\n38,https://www.redbus.in/bus-tickets/jahoo-to-delhi,Jahu to Delhi
305,HRTC - 1107,Ordinary 3+2 Non AC Seater,19:45,04:15,08h 30m,INR 560,36 Seats available,4.0,https://www.redbus.in/bus-tickets/jahoo-to-delhi,Jahu to Delhi


# Assam Bus Details

In [34]:
# read the csv file
df_7=pd.read_csv("df_AS.csv")
df_7

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
3,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
6,North Lakhimpur to Sibsagar (Assam),https://www.redbus.in/bus-tickets/north-lakhim...
7,Guwahati to Dhubri,https://www.redbus.in/bus-tickets/guwahati-to-...
8,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
9,Dhekiajuli to Guwahati,https://www.redbus.in/bus-tickets/dhekiajuli-t...


In [35]:
# retrive the bus details
driver_AS = webdriver.Chrome()
Bus_names_AS = []
Bus_types_AS = []
Start_Time_AS = []
End_Time_AS = []
Ratings_AS = []
Total_Duration_AS = []
Prices_AS = []
Seats_Available_AS = []
Route_names_AS = []
Route_links_AS = []
for i,r in df_7.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]
# Loop through each link
    driver_AS.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_AS.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
         
    scrolling = True
    while scrolling:
        old_page_source = driver_AS.page_source
        
        ActionChains(driver_AS).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_AS.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_AS.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_AS.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_AS.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_AS.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_AS.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_AS.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_AS.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_AS.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_AS.append(bus.text)
        Route_links_AS.append(link)
        Route_names_AS.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_AS.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_AS.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_AS.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_AS.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_AS.append(ratings.text)
    for price_elem in price:
        Prices_AS.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_AS.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [36]:
# from list to convert data frame
data_8 = {
    'Bus_name': Bus_names_AS,
    'Bus_type': Bus_types_AS,
    'Start_time': Start_Time_AS,
    'End_time': End_Time_AS,
    'Total_duration': Total_Duration_AS,
    'Price': Prices_AS,
    "Seats_Available":Seats_Available_AS,
    "Ratings":Ratings_AS,
    'Route_link': Route_links_AS,
    'Route_name': Route_names_AS
}

df_buses_8 = pd.DataFrame(data_8)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_8.csv"
df_buses_8.to_csv(path,index=False)

In [37]:
df_buses_8

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,NEELKANTH TRAVELS,A/C Seater Push Back (2+2),05:05,08:30,03h 25m,297,40 Seats available,4.5\n198,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
1,Kanchan Travels,AC Seater (2+2),05:20,08:30,03h 10m,INR 278,30 Seats available,4.4\n63,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
2,Mahi Travels(Under ASTC),A/C Seater (2+2),05:40,08:55,03h 15m,348,40 Seats available,4.6\n338,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
3,WARISPIYA TRAVELS,AC Seater (2+2),05:45,09:00,03h 15m,INR 360,35 Seats available,4.1\n336,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
4,Orient Transline,A/C Seater (2+1),05:55,09:00,03h 05m,INR 380.95,35 Seats available,4.3\n96,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
5,Mahi Travels(Under ASTC),Bharat Benz A/C Seater (2+2),06:10,09:40,03h 30m,311,35 Seats available,3.4\n91,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
6,Aziba Travels,NON A/C Seater (2+1),09:45,14:10,04h 25m,347,32 Seats available,4.0\n5,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
7,WARISPIYA TRAVELS,Bharat Benz A/C Seater (2+2),06:30,10:25,03h 55m,INR 360,36 Seats available,4.5\n346,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
8,Orient Transline,A/C Seater (2+1),06:45,10:10,03h 25m,INR 380.95,32 Seats available,4.8\n49,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
9,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),10:10,15:00,04h 50m,405,37 Seats available,3.2\n12,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati


# Uttra Pradesh bus details

In [38]:
# read the csv file
df_8=pd.read_csv("df_UP.csv")
df_8

,Route_name,Route_link
0,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
1,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
2,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
3,Lucknow to Delhi,https://www.redbus.in/bus-tickets/lucknow-to-d...
4,Lucknow to Prayagraj(Uttar Pradesh),https://www.redbus.in/bus-tickets/lucknow-to-a...
5,Farrukhabad (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/farrukhabad-...
6,Prayagraj(Uttar Pradesh) to Lucknow,https://www.redbus.in/bus-tickets/allahabad-to...
7,Sitapur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/sitapur-to-d...
8,Gorakhpur (uttar pradesh) to Lucknow,https://www.redbus.in/bus-tickets/gorakhpur-to...
9,Agra to Lucknow,https://www.redbus.in/bus-tickets/agra-to-lucknow


In [39]:
# retrive the bus details
driver_UP = webdriver.Chrome()
Bus_names_UP = []
Bus_types_UP = []
Start_Time_UP = []
End_Time_UP = []
Ratings_UP = []
Total_Duration_UP = []
Prices_UP = []
Seats_Available_UP = []
Route_names = []
Route_links = []

for i,r in df_8.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_UP.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_UP.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2) 
        
    scrolling = True
    while scrolling:
        old_page_source = driver_UP.page_source
        
        ActionChains(driver_UP).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_UP.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_UP.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_UP.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_UP.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_UP.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_UP.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_UP.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_UP.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_UP.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_UP.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_UP.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_UP.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_UP.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_UP.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_UP.append(ratings.text)
    for price_elem in price:
        Prices_UP.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_UP.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [40]:
# from list to convert data frame
data_9 = {
    'Bus_name': Bus_names_UP,
    'Bus_type': Bus_types_UP,
    'Start_time': Start_Time_UP,
    'End_time': End_Time_UP,
    'Total_duration': Total_Duration_UP,
    'Price': Prices_UP,
    "Seats_Available":Seats_Available_UP,
    "Ratings":Ratings_UP,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_9 = pd.DataFrame(data_9)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_9.csv"
df_buses_9.to_csv(path,index=False)

In [41]:
df_buses_9

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,RS YADAV SMART BUS,Bharat Benz A/C Seater /Sleeper (2+1),21:40,04:45,07h 05m,INR 999,30 Seats available,4.1\n50,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly (Uttar Pradesh)
1,New Khaira Transport,A/C Seater / Sleeper (2+1),20:40,03:30,06h 50m,630,28 Seats available,4.4\n16,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly (Uttar Pradesh)
2,New Khaira TPT,AC Sleeper (2+1),20:30,02:50,06h 20m,INR 900,27 Seats available,4.3\n13,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly (Uttar Pradesh)
3,SHRI RAM TRAVELS,A/C Seater (2+2),11:45,16:45,05h 00m,INR 740,15 Seats available,3.8\n15,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly (Uttar Pradesh)
4,Vijay Tour and Travels,A/C Seater / Sleeper (2+1),01:40,06:20,04h 40m,INR 749,34 Seats available,2.8\n70,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly (Uttar Pradesh)
...,...,...,...,...,...,...,...,...,...,...
305,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),02:00,06:00,04h 00m,INR 599,22 Seats available,3.7,https://www.redbus.in/bus-tickets/delhi-to-har...,Delhi to Haridwar
306,City Land Travels,A/C Sleeper (2+1),22:30,04:00,05h 30m,349,34 Seats available,4.3\n340,https://www.redbus.in/bus-tickets/delhi-to-har...,Delhi to Haridwar
307,AdE Bus India,A/C Seater/Sleeper (2+1),23:30,04:30,05h 00m,449,35 Seats available,4.2\n70,https://www.redbus.in/bus-tickets/delhi-to-har...,Delhi to Haridwar
308,Zingbus Plus,A/C Seater (2+2),23:00,04:00,05h 00m,352,37 Seats available,4.6\n100,https://www.redbus.in/bus-tickets/delhi-to-har...,Delhi to Haridwar


# West Bengal Bus Details

In [42]:
# read the csv file
df_9=pd.read_csv("df_WB.csv")
df_9

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
3,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
4,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
5,Barasat (West Bengal) to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/barasat-west...
6,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
7,Midnapore to Barasat (West Bengal),https://www.redbus.in/bus-tickets/midnapore-to...
8,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...
9,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...


In [43]:
# retrive the bus details
driver_WB = webdriver.Chrome()
Bus_names_WB = []
Bus_types_WB = []
Start_Time_WB = []
End_Time_WB = []
Ratings_WB = []
Total_Duration_WB = []
Prices_WB = []
Seats_Available_WB = []
Route_names = []
Route_links = []

for i,r in df_9.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]
# Loop through each link
    driver_WB.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_WB.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2) 
        
    scrolling = True
    while scrolling:
        old_page_source = driver_WB.page_source
        
        ActionChains(driver_WB).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_WB.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_WB.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_WB.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_WB.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_WB.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_WB.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_WB.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_WB.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_WB.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_WB.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_WB.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_WB.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_WB.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_WB.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_WB.append(ratings.text)
    for price_elem in price:
        Prices_WB.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_WB.append(seats_elem.text)
print("Successfully Completed")

Successfully Completed


In [44]:
# from list to convert data frame
data_10 = {
    'Bus_name': Bus_names_WB,
    'Bus_type': Bus_types_WB,
    'Start_time': Start_Time_WB,
    'End_time': End_Time_WB,
    'Total_duration': Total_Duration_WB,
    'Price': Prices_WB,
    "Seats_Available":Seats_Available_WB,
    "Ratings":Ratings_WB,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_10 = pd.DataFrame(data_10)
#convert dataframe to csv
path=r"C:\Users\User\Desktop\capstone project guvi\redbus_project\df_buses_10.csv"
df_buses_10.to_csv(path,index=False)

In [45]:
df_buses_10

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Pammi Travels,Non A/C Seater / Sleeper (2+1),02:00,06:00,04h 00m,INR 650,50 Seats available,3.6,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
1,Royal Cruiser,Volvo A/C Seater (2+2),02:00,06:00,04h 00m,463,23 Seats available,3.6\n85,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
2,Pammi Travels,NON A/C Sleeper (2+1),02:00,06:00,04h 00m,INR 750,22 Seats available,3.6,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
3,Express Line (Karunamoyee),Volvo 9600 Multi Axle Semi-Sleeper (2+2),06:10,09:20,03h 10m,INR 486,27 Seats available,4.5\n11,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
4,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),06:10,09:20,03h 10m,INR 486,44 Seats available,4.0\n7,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
5,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),06:10,09:20,03h 10m,INR 486,44 Seats available,4.4\n198,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
6,Express Line (Karunamoyee),Volvo 9600 Multi Axle Semi-Sleeper (2+2),06:10,09:20,03h 10m,INR 486,27 Seats available,4.6\n273,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
7,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),06:20,09:30,03h 10m,INR 486,41 Seats available,3.8\n7,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
8,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),06:20,09:30,03h 10m,INR 486,41 Seats available,4.2\n310,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
9,Greenline,Volvo A/C Seater (2+2),06:30,09:45,03h 15m,INR 486,28 Seats available,4.3\n213,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur (West Bengal) to Kolkata
